<a href="https://colab.research.google.com/github/domingues100/IEEE---Water_Level/blob/main/OD_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inicialization**

Install detectron2 and clone cocosplit

-Restart environment after that

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!git clone https://github.com/akarazniewicz/cocosplit.git
!pip install funcy
!pip install scikit-multilearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# TRAIN MODEL

***Libraries***

In [ ]:
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.data import transforms as T
from detectron2.data.datasets import register_coco_instances, load_coco_json
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from google.colab.patches import cv2_imshow
import cv2
import os
import shutil

***Define some augmentations***

In [ ]:
class CustomAugmentation(T.Augmentation):
    def get_transform(self, image):
        transforms = [
            T.RandomFlip(prob=0.5, horizontal=True),
            T.RandomBrightness(0.1, 0.5),
            T.RandomContrast(0.1, 0.5),
            T.RandomRotation(angle=[-25, 25]),
            T.RandomNoise(intensity=0.12),
            T.RandomZoom(scale=(0.1, 0.3)),
            T.ElasticTransform(alpha=1.2, sigma=0.2),
            T.CropAndPad(percent=(0, 0.4))
        ]
        return T.TransformList(transforms)

***Split Coco***

In [ ]:
#name of the test
test = "test_fasterrcnn"

#split test and train using cocosplit
!python cocosplit/cocosplit.py -s 0.7 "/content/drive/MyDrive/IC_Machine_Learning/instances_default.json" train.json test.json

#move train and test to desired folders
archives = ['train.json', 'test.json']

for archive in archives:
    os.makedirs(f"/content/drive/MyDrive/Transaciones/{test}", exist_ok=True)
    origin_path = os.path.join("/content/", archive)
    final_path = os.path.join(f"/content/drive/MyDrive/Transaciones/{test}", archive)

    shutil.copy(origin_path, final_path)

In [ ]:
test_json = f"/content/drive/MyDrive/Transaciones/{test}/test.json"
train_json = f"/content/drive/MyDrive/Transaciones/{test}/train.json"
images = "/content/drive/MyDrive/IC_Machine_Learning/images"

register_coco_instances(f"test_dataset_no_crop_{teste}", {}, test_json, images)
register_coco_instances(f"train_dataset_no_crop_{teste}", {}, train_json, images)

DatasetCatalog.register("test_dataset", lambda: register_coco_instances(f"test_dataset_no_crop_{teste}", {}, test_json, images), )
DatasetCatalog.register("train_dataset", lambda: register_coco_instances(f"train_dataset_no_crop_{teste}", {}, train_json, images))

teste_dataset_no_crop_metadata = MetadataCatalog.get(f"teste_dataset_no_crop_{test}")
train_dataset_no_crop_metadata = MetadataCatalog.get(f"train_dataset_no_crop_{test}")

In [ ]:
import gc
gc.collect()

57

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (f"train_dataset_no_crop_{teste}",)
cfg.DATASETS.TEST = (f"teste_dataset_no_crop_{teste}",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 250
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (128)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.INPUT.MAX_SIZE_TRAIN = 384
cfg.INPUT.MIN_SIZE_TEST = 384
cfg.INPUT.MAX_SIZE_TEST = 384

cfg.INPUT.AUGMENTATIONS = [("CustomAugmentation", CustomAugmentation())]

OUTPUT_DIR = f"/content/drive/MyDrive/Transaciones/{teste}"
os.makedirs(OUTPUT_DIR, exist_ok=True)
cfg.OUTPUT_DIR = OUTPUT_DIR

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

***Evaluate the perfomance***

In [ ]:
evaluator = COCOEvaluator(f"test_dataset_no_crop_{test}", cfg, False, output_dir=f"/content/drive/MyDrive/Transaciones/{test}")
val_loader = build_detection_test_loader(cfg, f"test_dataset_no_crop_{test}")
print(inference_on_dataset(trainer.model, val_loader, evaluator))

# Load Model and test/calculate IoU


In [ ]:
test = "teste_fasterrcnn"

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.checkpoint import DetectionCheckpointer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")) #FOR FASTERCNN
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = f"/content/drive/MyDrive/Transaciones/{teste}/model_final.pth"
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 250
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (128)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.INPUT.RANDOM_FLIP = "horizontal"
cfg.INPUT.MAX_SIZE_TRAIN = 384
cfg.INPUT.MIN_SIZE_TEST = 384
cfg.INPUT.MAX_SIZE_TEST = 384
cfg.INPUT.AUGMENTATIONS = [("CustomAugmentation", CustomAugmentation())]

model = DefaultPredictor(cfg).model
checkpointer = DetectionCheckpointer(model)
checkpointer.load(cfg.MODEL.WEIGHTS)


In [ ]:
test_json = "/content/drive/MyDrive/Transaciones/test_fasterrcnn/train.json"
images = "/content/drive/MyDrive/IC_Machine_Learning/images"

In [ ]:
register_coco_instances("test_dataset_rcnn", {}, test_json, images)

DatasetCatalog.register("test_dataset", lambda: register_coco_instances("test_dataset_rcnn", {}, test_json, images), )

teste_dataset_no_crop_metadata = MetadataCatalog.get("test_dataset_rcnn")

***Para usar o evaluator***

In [ ]:
evaluator = COCOEvaluator("test_dataset_rcnn", cfg, False, output_dir=f"/content/drive/MyDrive/IC_Machine_Learning/Deteccao_detectron2/{test}")
val_loader = build_detection_test_loader(cfg, "test_dataset_rcnn")
print(inference_on_dataset(model, val_loader, evaluator))

***Predictions and Crop BBOX***

In [ ]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0
predictor = DefaultPredictor(cfg)
best_predictions = []
gt_bbox = []
names = []
teste_dataset_dicts = load_coco_json(test_json, images)

output_images_dir = f"/content/drive/MyDrive/Transaciones/Images_RCNN"
os.makedirs(output_images_dir, exist_ok=True)

for d in teste_dataset_dicts:
    im = cv2.imread(d["file_name"])

    #saving gt bbox
    gt_bbox.append(d["annotations"][0]["bbox"])
    names.append(d["file_name"])
    outputs = predictor(im)

    detections = outputs["instances"].to("cpu")

    if len(detections) > 0:
        scores = detections.scores.tolist()
        max_score_idx = scores.index(max(scores))
        best_detection = detections[max_score_idx]
        bbox = best_detection.pred_boxes.tensor.cpu().numpy()[0]

        #saving best_prediction bbox
        best_predictions.append(bbox)
        x, y, w, h = bbox
        x, y, w, h = int(x), int(y), int(w), int(h)

        cropped_image = im[y:h, x:w]

        # Salvar o recorte
        output_image_path = os.path.join(output_images_dir, os.path.basename(d["file_name"]))
        cv2.imwrite(output_image_path, cropped_image)

***IOU***

In [ ]:
def gt_bbox_coco_format(gt_bbox):
  gt_bbox_copy = gt_bbox.copy()
  for item in gt_bbox_copy:
    x, y, w, h = item
    item[2] = x+w
    item[3] = y+h
  return gt_bbox_copy

def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) between two bounding boxes.

    Parameters:
    - box1: Tuple (x1, y1, x2, y2) representing the coordinates of the first bounding box.
    - box2: Tuple (x1, y1, x2, y2) representing the coordinates of the second bounding box.

    Returns:
    - IoU: Intersection over Union value.
    """

    # Calculate the intersection coordinates
    x1_i = max(box1[0], box2[0])
    y1_i = max(box1[1], box2[1])
    x2_i = min(box1[2], box2[2])
    y2_i = min(box1[3], box2[3])

    # Calculate the intersection area
    intersection_area = max(0, x2_i - x1_i + 1) * max(0, y2_i - y1_i + 1)

    # Calculate the union area
    area_box1 = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    area_box2 = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    union_area = area_box1 + area_box2 - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area

    # append iou in a list
    iou_list.append(iou)


#list to store iou
iou_list = []

formated_gt_bbox = gt_bbox_coco_format(gt_bbox)

for item1, item2 in zip(formated_gt_bbox, best_predictions):
  calculate_iou(item1, item2)

mean_iou = sum(iou_list)/len(iou_list)
print(mean_iou)
